### Практическая работа к уроку №6

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import (roc_auc_score, precision_recall_curve)
from sklearn.ensemble import RandomForestClassifier

#### 1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

https://archive.ics.uci.edu/ml/datasets/YouTube+Spam+Collection

YouTube Spam Collection Data Set

In [2]:
datasets_path = [
    "YouTube-Spam-Collection-v1/Youtube01-Psy.csv",
    "YouTube-Spam-Collection-v1/Youtube02-KatyPerry.csv",
    "YouTube-Spam-Collection-v1/Youtube03-LMFAO.csv",
    "YouTube-Spam-Collection-v1/Youtube04-Eminem.csv",
    "YouTube-Spam-Collection-v1/Youtube05-Shakira.csv"
]

df = pd.concat([pd.read_csv(path_df) for path_df in datasets_path])

2. сделать feature engineering
3. обучить любой классификатор (какой вам нравится)

In [3]:
df.columns = [column.capitalize() for column in df.columns]
df.head()

,Comment_id,Author,Date,Content,Class
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [4]:
#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]

classifier = Pipeline([('content_selector', FeatureSelector(column='Content')), 
                     ('content_text_tfidf', TfidfVectorizer(sublinear_tf=True,
                                                            strip_accents='unicode',
                                                            analyzer='word',
                                                            token_pattern=r'\w{1,}',
                                                            stop_words='english',
                                                            ngram_range=(1, 1),
                                                            max_features=10000)), 
                     ('classifier', RandomForestClassifier(random_state = 42))])

In [5]:
def get_metrics(y_true, y_pred):
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred)

    fscore = (2 * precision * recall) / (precision + recall)
    ix = np.argmax(fscore)
    
    return fscore[ix], precision[ix], recall[ix], roc_auc_score(y_true, y_pred), thresholds[ix]

In [6]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('Class', 1), 
                                                    df['Class'], random_state=0)

In [7]:
#обучим пайплайн на всем тренировочном датасете
classifier.fit(X_train, y_train)
y_pred = classifier.predict_proba(X_test)[:, 1]

results = {}

results['RandomForest'] = get_metrics(y_test, y_pred)

#### 4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
#### 5. применить random negative sampling для построения классификатора в новых условиях
#### 6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
#### 7. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

In [8]:
# доли P
sizes_P = np.arange(0.2, 1.01, 0.05)

for size_P in sizes_P:
    mod_data = df.copy()
    
#     Извлечем size_P долю объектов с таргетом 1 из датасета
    pos_ind = np.where(mod_data.loc[:,'Class'].values == 1)[0]
    pos_sample = np.random.choice(pos_ind, replace=True, size=int(size_P * len(pos_ind)))
    
#     Заведем новый столбец с фейковым классом
    mod_data['fake_class'] = 0
    mod_data.iloc[pos_sample, -1] = 1
    
#     Извлечем из датасета множество P.
#     Из множества U извлечем множество N такого же размера как и P
#     оставшиеся объекты останутся для валидации
    mod_data = mod_data.sample(frac=1)
    condition_true = (mod_data['fake_class'] == 1)
    condition_false = (mod_data['fake_class'] == 0)
    size_data_P = sum(mod_data['fake_class'] == 1)

    data_N = mod_data[condition_false][:size_data_P]
    data_U = mod_data[condition_false][size_data_P:]
    data_P = mod_data[condition_true]
    if data_N.shape != data_P.shape:
        print('error')
    train_data = pd.concat([data_N, data_P]).sample(frac=1)
    
#     Обучение модели
    classifier.fit(train_data.iloc[:,:-2],
                   train_data.iloc[:,-1])
#     Предикт на множестве U
    y_pred = classifier.predict_proba(data_U.iloc[:,:-2])[:, 1]
    results[round(size_P, 2)] = get_metrics(data_U.iloc[:,-2], y_pred)

In [9]:
result_df = pd.DataFrame.from_dict(results, orient="index",
                       columns=['fscore', 'precision',
                                'recall', 'roc_auc_score', 'threshold'])
result_df = result_df.sort_values(by='fscore', ascending=False).round(3)
result_df

,fscore,precision,recall,roc_auc_score,threshold
RandomForest,0.952,0.969,0.935,0.977,0.480
1.0,0.937,0.939,0.935,0.983,0.442
0.9,0.936,0.971,0.904,0.978,0.445
0.85,0.925,0.974,0.881,0.973,0.477
0.75,0.920,0.926,0.914,0.978,0.380
0.55,0.919,0.936,0.902,0.976,0.360
0.95,0.919,0.941,0.897,0.979,0.420
0.6,0.918,0.910,0.926,0.981,0.385
0.5,0.912,0.911,0.913,0.968,0.375
0.7,0.910,0.894,0.926,0.975,0.340


Результаты получились вполне логичными. Чем меньшую долю P мы отбираем из датасета, тем меньшее качество модели мы получаем